<a href="https://colab.research.google.com/github/mandrecki/fun_notebooks/blob/main/windTurbineAllocation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python
# coding: utf-8

# In[1]:


import numpy as np
import numpy.matlib
import time
import matplotlib.pyplot as plt
from IPython import display
# Define parameters

# Starting point
nObj = 10; #Number of turbines
#startCoord = np.array([0.5, 0.5]); #Square area
startCoord = np.array([498, 580])/1000; #POLSKA !!
initDist = 0.05; # Initial radius around the centre point
angleSeed = np.linspace(0, 2*np.pi, nObj, endpoint = False); #Distribute points around the centre point
# Generate initial position
objCoords = np.array([initDist*np.cos(angleSeed)+startCoord[0], initDist*np.sin(angleSeed)+startCoord[1]]);
objVel = np.zeros((2, nObj)); #Prealloate velocities

# Bounded area
#edgeNodes = np.array([[0, 0], [1, 0], [1, 1], [0, 1]]); # Square
edgeNodes = np.array([[57, 145], [70, 627], [453, 873], [855, 945], [974, 682], [881, 102], [462, 104], [448, 27]])/1000; #POLSKA !!
edgeNodes = np.append(edgeNodes, [edgeNodes[0,:]], axis=0); # Add the first point to loop the shape
edgeParams = numpy.empty((3, edgeNodes.shape[0]-1)); #Preallocate the equation of edge normal
edgeDirVec = numpy.empty((2, edgeNodes.shape[0]-1)); #Not needed really
for ii in range(0, edgeParams.shape[1]): #Loop for every edge
    edgeDirVec[:,ii] = np.array([edgeNodes[ii+1,0]-edgeNodes[ii,0], edgeNodes[ii+1,1]-edgeNodes[ii,1]]); #Find the equation of the edge segment
    k = edgeDirVec[0,ii]*edgeNodes[ii,1]-edgeDirVec[1,ii]*edgeNodes[ii,0] #Find the constant term in the normal equation
    edgeParams[:,ii] = [edgeDirVec[1,ii], -edgeDirVec[0,ii], k]/(edgeDirVec[1,ii]**2+edgeDirVec[0,ii]**2)**0.5; #Generate equation of normal ax+by+k = 0
    if np.cross(edgeDirVec[:,ii], edgeParams[0:2,ii]) > 0: #Check if normal if facing inwards of the bounded area
        edgeParams[0:2, ii] = -edgeParams[0:2, ii];

#print(edgeParams[0:2,:])

# Simulation parameters
dt = 0.01; #integration time
tMax = 15; #Max time of simulation
t = 0.0; #Current time
m = 1000.0; #Mass of particles
alpha = 200.0; #Relaxation time of velocity (artificial damping)







In [ ]:
# Calculate distances between all turbines
def calculateDistanceToOthers(objNum, objsCoords):
    #Add valid indicies (remove self distance)
    validIdx = np.arange(objsCoords.shape[1]);
    validIdx = np.delete(validIdx, objNum);
    directionVector = np.array([objsCoords[0, objNum]-objsCoords[0,validIdx], objsCoords[1, objNum]-objsCoords[1,validIdx]]);
    distance = (directionVector[0,:]**2+directionVector[1,:]**2)**0.5;
    directionVersor = np.divide(directionVector, np.matlib.repmat(distance, 2, 1));
    return distance, directionVersor

#Calculate distance from one turbine to all edges
def calculateDistanceToEdges(objCoords, edgeNodes):
    # Algo from https://stackoverflow.com/questions/849211/shortest-distance-between-a-point-and-a-line-segment
    edgeDir = np.diff(edgeNodes,axis=0);
    l2 = edgeDir[:,0]**2+edgeDir[:,1]**2;
    # If the point is closest to the line section of the edge
    edgeSectionDomain = np.divide(np.einsum("ij,ij->i", np.matlib.repmat(objCoords,edgeNodes.shape[0]-1,1) - edgeNodes[:-1,:], edgeNodes[1:,:] - edgeNodes[:-1,:]), l2);
    # Check where the point projects to
    beyondDistance = np.amin(np.concatenate((np.ones((edgeNodes.shape[0]-1,)), edgeSectionDomain)).reshape(2,-1), axis=0);
    t = np.amax(np.concatenate((np.zeros(edgeNodes.shape[0]-1,), beyondDistance)).reshape(2,-1), axis=0);
    projection = edgeNodes[:-1,:] + np.array([t,t]).transpose() *  (edgeNodes[1:,:] - edgeNodes[:-1,:]);
    distance = ((objCoords[0]-projection[:,0])**2+(objCoords[1]-projection[:,1])**2)**0.5;
    return distance

#Calculate acting force
def calculateActingForce(distObjs, directObjs, distEdge, edgeParams):
    #Between objects
    magObjs = 1/distObjs**2;
    FxObjs = np.multiply(directObjs[0,:], magObjs);
    FyObjs = np.multiply(directObjs[1,:], magObjs);
    #Between object and edges
    magEdges = 0.01/distEdge**4;
    dirEdges = edgeParams[0:2,:];
    FxEdges = np.multiply(dirEdges[0,:], magEdges);
    FyEdges = np.multiply(dirEdges[1,:], magEdges);
    force = np.array([FxObjs.sum()+FxEdges.sum(), FyObjs.sum()+FyEdges.sum()]);
    return force

In [ ]:
# Plot original state
fig, ax = plt.subplots()
plt.ion()
ax.set_aspect('equal', adjustable='box')

# Plot bounding area
for ii in range(0,edgeParams.shape[1]):
    ax.plot([edgeNodes[ii][0], edgeNodes[ii+1][0]], [edgeNodes[ii][1], edgeNodes[ii+1][1]], linewidth=2.0)  

# Plot original seeds
plot1,  = ax.plot(objCoords[0,:], objCoords[1,:],ms=10,color='k',marker='o',ls='');
ax.axis('equal')
#fig
#plt.show();
plt.xlim([0, 1]);
plt.ylim([0, 1]);
ax.invert_yaxis()
# Loop until the end of simulation
k = 0; # Plot update numerator
while t<tMax:
    # Calculate force for each wind turbine (should be vectorised in future)
    for ii in range(0, nObj):
        r, direct = calculateDistanceToOthers(ii, objCoords); #Get distance and direction to all other turbines
        rEdge = calculateDistanceToEdges(objCoords[:, ii], edgeNodes); #Get distance to the edges
        F = calculateActingForce(r, direct, rEdge, edgeParams); #Calculate the forces
        objVel[:,ii] = objVel[:,ii] + F/m*dt; #Update the velocity through acceleration
    objVel = objVel*np.exp(-dt/alpha); #Add damping to velocity to eventually remove kinematic energy
    objCoords = objCoords + objVel*dt; #Update positions of all particles
    t += dt; #Increment time
    #Update plots
    if (k % 50) == 0:
        plot1.set_xdata(objCoords[0,:])
        plot1.set_ydata(objCoords[1,:])
        display.display(plt.gcf())
        display.clear_output(wait=True)

        #time.sleep(1);